In [1]:
import pandas as pd
df1=pd.read_parquet('data_vk_hse/0000.parquet', engine='fastparquet')
df2=pd.read_parquet('data_vk_hse/0001.parquet', engine='fastparquet')
df3=pd.read_parquet('data_vk_hse/0002.parquet', engine='fastparquet')
df4=pd.read_parquet('data_vk_hse/0003.parquet', engine='fastparquet')
df5=pd.read_parquet('data_vk_hse/0004.parquet', engine='fastparquet')
df6=pd.read_parquet('data_vk_hse/0005.parquet', engine='fastparquet')
df7=pd.read_parquet('data_vk_hse/0006.parquet', engine='fastparquet')
df8=pd.read_parquet('data_vk_hse/0007.parquet', engine='fastparquet')
df9=pd.read_parquet('data_vk_hse/0008.parquet', engine='fastparquet')
df10=pd.read_parquet('data_vk_hse/0009.parquet', engine='fastparquet')

In [2]:
df = pd.concat ([df1, df2, df3,df4,df5,df6,df7,df8,df9,df10])
feature_to_drop=['title','blocks.data','username','author_id','timestamp', 'id','blocks.type','comments.id','comments.timestamp','comments.parent_id','comments.text_markdown','comments.text_html','comments.images','comments.rating','comments.pluses',	'comments.minuses','comments.author_id','comments.username','rating','pluses','minuses','url']
df.drop(feature_to_drop, axis=1, inplace=True)
df = df.explode('tags')
import catboost as cb
df = df.sample(frac=0.0001) # Get 30% of the data
len(df)  

114

In [3]:
df

,text_markdown,tags
14729,,Комиксы
5633,,Км
9962,План эвакуации в пермской галерее.\nВ 2020 год...,Фрейд
7659,,Бег
21616,"1\. Зима. Первый снег.\nКак всегда, пролежал о...",Жизнь
...,...,...
13668,4 февраля. Демонстрация обманутых вкладчиков п...,Выборы
14195,"НИКОГДА, СЛЫШИТЕ?НИКОГДА НЕ ДАВАЙТЕ ДРУЗЬЯМ ДО...",Dota
22971,,Политика
30813,"UPD!!!!\nКак только отправила пост, пришло оче...",Сотовые операторы


In [22]:
# Предобработка текста
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

# Разделение данных на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(df[['text_markdown']], df['tags'], test_size=0.2, random_state=42)

# Создание модели CatBoost с указанием текстового признака
model = CatBoostClassifier(
    depth=6,
    iterations=10,
    learning_rate=0.06,
    verbose=10,
    text_features=['text_markdown']
)

# Обучение модели
model.fit(X_train, y_train)

# Предсказания
predictions = model.predict(X_test)

# Оценка модели
print("Accuracy:", accuracy_score(y_test, predictions))

0:	learn: 4.3414403	total: 1.06s	remaining: 14m 5s
10:	learn: 4.1881646	total: 11.5s	remaining: 13m 48s


KeyboardInterrupt: 

In [11]:
def predict_tag(text):
    # Подготавливаем входные данные
    input_df = pd.DataFrame({'text_markdown': [text]})
    
    # Предсказываем
    predicted_tag = model.predict(input_df)
    
    return predicted_tag[0]  # Возвращает первый элемент, так как результатом будет одиночный тег

# Пример использования
predicted_tag = predict_tag('В Иране  началсь война тк слуйчайно выпустили не туда ракету')
print("Предсказанный тег:", predicted_tag)


Предсказанный тег: ['Текст']


In [32]:
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc, f1_score, roc_auc_score, accuracy_score
from sklearn.preprocessing import label_binarize
import numpy as np
from catboost import CatBoostClassifier

def load_catboost_model(filename):
    model = CatBoostClassifier()
    model.load_model(filename, format='cbm')
    return model
model = load_catboost_model('catboost_model.cbm')   

def plot_metrics(model, X_test, y_test):
    if hasattr(model, "predict_proba"):
        # Предсказываем вероятности
        y_scores = model.predict_proba(X_test)

        # Получаем список уникальных классов из y_test
        unique_classes = np.unique(y_test)
        y_test_bin = label_binarize(y_test, classes=unique_classes)
        n_classes = y_test_bin.shape[1]

        # Вычисляем микро-усредненные значения FPR и TPR
        if n_classes == 2:
            fpr, tpr, _ = roc_curve(y_test_bin, y_scores[:, 1])
        else:
            fpr, tpr, _ = roc_curve(y_test_bin.ravel(), y_scores.ravel())
        roc_auc = auc(fpr, tpr)

        # Визуализация общей ROC кривой
        plt.figure(figsize=(10, 8))
        plt.plot(fpr, tpr, label='Micro-average ROC curve (area = {:.2f})'.format(roc_auc))
        plt.plot([0, 1], [0, 1], 'k--')
        plt.xlim([0.0, 1.0])
        plt.ylim([0.0, 1.05])
        plt.xlabel('False Positive Rate')
        plt.ylabel('True Positive Rate')
        plt.title('Receiver Operating Characteristic')
        plt.legend(loc="lower right")
        plt.show()

    # F1 Score и точность
    y_pred = model.predict(X_test)
    f1 = f1_score(y_test, y_pred, average='weighted')
    accuracy = accuracy_score(y_test, y_pred)
    print("F1 Score:", 1-f1)
    print("Accuracy:", 1-accuracy)

plot_metrics(model, X_test, y_test)

In [13]:
def show_predicted_tags_with_probabilities(model, X_test):
    # Проверяем, поддерживает ли модель predict_proba
    if hasattr(model, "predict_proba"):
        probabilities = model.predict_proba(X_test)
        class_labels = model.classes_
        for index, prob in enumerate(probabilities):
            print(f"Sample {index + 1}:")
            for class_index, class_prob in enumerate(prob):
                print(f"   {class_labels[class_index]}: {class_prob:.4f}")
            print("\n")
    else:
        print("This model does not support probability predictions.")


In [9]:
def save_catboost_model(model, filename):
    model.save_model(filename, format='cbm')

# Пример использования:
save_catboost_model(model, 'catboost_model.cbm')


In [ ]:
from catboost import CatBoostClassifier

def load_catboost_model(filename):
    model = CatBoostClassifier()
    model.load_model(filename, format='cbm')
    return model

# Пример использования:
# loaded_catboost_model = load_catboost_model('catboost_model.cbm')


In [ ]:
напиши также пожалуйста нейронную сеть на pytorch которая будет обучаться на тексте и таргет бует в виде списка слов в итоге она должна выводить класс с наибольшей вероятностью остальный классы с вероятностями график метрики roc_auc и f1

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score, f1_score, roc_curve, auc


In [ ]:
# Предполагаем, что `model` - ваша обученная модель CatBoost, а `X_test` и `y_test` - тестовые данные

# Предсказания вероятностей
y_probs = model.predict_proba(X_test)[:, 1]  # Извлечение вероятностей для положительного класса

# Расчет ROC AUC
roc_auc = roc_auc_score(y_test, y_probs)
print(f"ROC AUC Score: {roc_auc:.2f}")

# Бинаризация предсказаний для F1-score (порог можно настроить)
y_pred = (y_probs >= 0.5).astype(int)
f1 = f1_score(y_test, y_pred)
print(f"F1 Score: {f1:.2f}")


In [ ]:
fpr, tpr, _ = roc_curve(y_test, y_probs)
roc_auc = auc(fpr, tpr)

plt.figure()
plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic')
plt.legend(loc="lower right")
plt.show()


In [24]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from sklearn.metrics import roc_auc_score, f1_score
import numpy as np

# Создание датасета
class TextDataset(Dataset):
    def __init__(self, texts, labels):
        self.texts = texts
        self.labels = labels

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        return self.texts[idx], self.labels[idx]

# Определение модели
class TextClassifier(nn.Module):
    def __init__(self, vocab_size, num_classes):
        super(TextClassifier, self).__init__()
        self.embedding = nn.Embedding(vocab_size, 128)
        self.conv = nn.Conv1d(128, 256, 5)
        self.pool = nn.MaxPool1d(5)
        self.fc = nn.Linear(256, num_classes)
        self.activation = nn.Sigmoid()

    def forward(self, x):
        x = self.embedding(x)
        x = x.permute(0, 2, 1)
        x = self.conv(x)
        x = self.pool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)
        return self.activation(x)

# Инициализация модели, оптимизатора и критерия
model = TextClassifier(vocab_size=10000, num_classes=20)
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.BCELoss()

# Обучение
def train(model, dataloader, optimizer, criterion, epochs):
    model.train()
    for epoch in range(epochs):
        for texts, labels in dataloader:
            optimizer.zero_grad()
            outputs = model(texts)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

# Оценка
def evaluate(model, dataloader):
    model.eval()
    all_labels = []
    all_preds = []
    with torch.no_grad():
        for texts, labels in dataloader:
            outputs = model(texts)
            all_labels.extend(labels.numpy())
            all_preds.extend(outputs.numpy())

    roc_auc = roc_auc_score(all_labels, all_preds, average='macro')
    f1 = f1_score(all_labels, np.round(all_preds), average='macro')
    return roc_auc, f1

# Дополните данными и дополнительной логикой для демонстрации


In [ ]:

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from sklearn.metrics import roc_auc_score, f1_score
import numpy as np
import matplotlib.pyplot as plt

# Создание датасета
class TextDataset(Dataset):
    def __init__(self, texts, labels):
        self.texts = texts
        self.labels = torch.Tensor(labels)

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        return self.texts[idx], self.labels[idx]

# Определение модели
class TextClassifier(nn.Module):
    def __init__(self, vocab_size, num_classes):
        super(TextClassifier, self).__init__()
        self.embedding = nn.Embedding(vocab_size, 128)
        self.conv = nn.Conv1d(128, 256, 5)
        self.pool = nn.MaxPool1d(5)
        self.fc = nn.Linear(256, num_classes)
        self.activation = nn.Sigmoid()

    def forward(self, x):
        x = self.embedding(x)
        x = x.permute(0, 2, 1)
        x = self.conv(x)
        x = self.pool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)
        return self.activation(x)

# Инициализация модели, оптимизатора и критерия
model = TextClassifier(vocab_size=10000, num_classes=20)
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.BCELoss()

# Имитация данных
num_samples = 1000
vocab_size = 10000
num_classes = 20
max_length = 50

texts = torch.randint(0, vocab_size, (num_samples, max_length))
labels = torch.randint(0, 2, (num_samples, num_classes)).float()

dataset = TextDataset(texts, labels)
dataloader = DataLoader(dataset, batch_size=10, shuffle=True)

# Функция обучения
def train(model, dataloader, optimizer, criterion, epochs):
    model.train()
    for epoch in range(epochs):
        for texts, labels in dataloader:
            optimizer.zero_grad()
            outputs = model(texts)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
        print(f'Epoch {epoch+1}, Loss: {loss.item()}')

# Функция оценки
def evaluate(model, dataloader):
    model.eval()
    all_labels = []
    all_preds = []
    with torch.no_grad():
        for texts, labels in dataloader:
            outputs = model(texts)
            all_labels.append(labels)
            all_preds.append(outputs)
    all_labels = torch.cat(all_labels).numpy()
    all_preds = torch.cat(all_preds).numpy()
    roc_auc = roc_auc_score(all_labels, all_preds, average='macro')
    f1 = f1_score(all_labels, (all_preds > 0.5).astype(int), average='macro')
    return roc_auc, f1

# Обучение и оценка
train(model, dataloader, optimizer, criterion, epochs=5)
roc_auc, f1 = evaluate(model, dataloader)
print(f'ROC AUC: {roc_auc}, F1 Score: {f1}')

# Визуализация результатов
def plot_metrics(roc_auc, f1):
    metrics = [roc_auc, f1]
    labels = ['ROC AUC', 'F1 Score']
    plt.bar(labels, metrics, color=['blue', 'green'])
    plt.ylabel('Score')
    plt.title('Performance Metrics')
    plt.ylim(0, 1)
    plt.show()

plot_metrics(roc_auc, f1)


In [ ]:

from sklearn.preprocessing import MultiLabelBinarizer
from torch.utils.data import DataLoader, Dataset
import numpy as np
import torch

# Предположим, что texts и tags - это ваши данные
texts = [
    "Что можно купить в Китае за цену нового iPhone...",
    "Осенью в России стартовали продажи очередной модели..."
]
tags = [
    ["Китай", "AliExpress", "Китайские товары", "Бизнес"],
    ["Россия", "Продажи", "Смартфоны"]
]

# Токенизация и преобразование текстов в индексы
vocab = {word: idx for idx, word in enumerate(set(' '.join(texts).split()))}
indexed_texts = [[vocab[word] for word in text.split()] for text in texts]

# Паддинг текстов до одинаковой длины
max_len = max(len(text) for text in indexed_texts)
padded_texts = [text + [0] * (max_len - len(text)) for text in indexed_texts]

# Преобразование тегов в бинарные векторы
mlb = MultiLabelBinarizer()
binary_tags = mlb.fit_transform(tags)

# Создание датасета
class TextTagDataset(Dataset):
    def __init__(self, texts, tags):
        self.texts = torch.LongTensor(texts)
        self.tags = torch.FloatTensor(tags)

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        return self.texts[idx], self.tags[idx]

# Создание DataLoader
dataset = TextTagDataset(padded_texts, binary_tags)
dataloader = DataLoader(dataset, batch_size=2, shuffle=True)

# Пример использования DataLoader
for text, tag in dataloader:
    print("Text batch:", text)
    print("Tag batch:", tag)


In [ ]:

import torch
from torch.utils.data import DataLoader, Dataset
from torch.nn.utils.rnn import pad_sequence
from sklearn.preprocessing import MultiLabelBinarizer
import numpy as np
from collections import Counter

# Пример данных
texts = [
    "Что можно купить в Китае за цену нового iPhone...",
    "Осенью в России стартовали продажи очередной модели..."
]
tags = [
    ["Китай", "AliExpress", "Китайские товары", "Бизнес"],
    ["Россия", "Продажи", "Смартфоны"]
]

# Токенизация текста
def tokenize(texts):
    tokenized_texts = [text.lower().split() for text in texts]
    word_counts = Counter(word for text in tokenized_texts for word in text)
    vocab = {word: i + 1 for i, word in enumerate(word_counts)}  # Индекс 0 зарезервирован для паддинга
    indexed_texts = [[vocab[word] for word in text] for text in tokenized_texts]
    return vocab, indexed_texts

vocab, indexed_texts = tokenize(texts)
padded_texts = pad_sequence([torch.tensor(text) for text in indexed_texts], batch_first=True, padding_value=0)

# Динамическое преобразование тегов
mlb = MultiLabelBinarizer()
binary_tags = mlb.fit_transform(tags)

# Создание датасета
class TextTagDataset(Dataset):
    def __init__(self, texts, tags):
        self.texts = texts
        self.tags = torch.FloatTensor(tags)

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        return self.texts[idx], self.tags[idx]

dataset = TextTagDataset(padded_texts, binary_tags)
dataloader = DataLoader(dataset, batch_size=2, shuffle=True)

# Вывод данных из DataLoader
for text, tag in dataloader:
    print("Text batch:", text)
    print("Tag batch:", tag)


In [ ]:
import torch
from torchtext.data import Field, TabularDataset, BucketIterator
import pandas as pd
import os

# Сохраняем DataFrame в CSV файл, чтобы использовать с torchtext
df.to_csv('data.csv', index=False)

# Определяем поля
TEXT = Field(sequential=True, tokenize='spacy', lower=True)
LABEL = Field(sequential=False, is_target=True)

# Создаем датасет
fields = {'text': ('text', TEXT), 'tags': ('label', LABEL)}
dataset = TabularDataset(path='data.csv', format='csv', fields=fields)

# Построим словарь
TEXT.build_vocab(dataset, max_size=10000, min_freq=2)
LABEL.build_vocab(dataset)

# Итераторы для обучения
batch_size = 32
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
train_iterator, valid_iterator = BucketIterator.splits(
    (dataset, dataset), batch_size=batch_size, device=device, sort_within_batch=True
)


In [ ]:
import torch.nn as nn
import torch.optim as optim

class TextClassifier(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim):
        super(TextClassifier, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.fc = nn.Linear(embedding_dim, hidden_dim)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_dim, output_dim)
    
    def forward(self, text):
        embedded = self.embedding(text)
        embedded = embedded.mean(dim=0)  # Среднее значение эмбеддингов
        hidden = self.relu(self.fc(embedded))
        output = self.fc2(hidden)
        return output

# Инициализация модели
vocab_size = len(TEXT.vocab)
embedding_dim = 100
hidden_dim = 256
output_dim = len(LABEL.vocab)

model = TextClassifier(vocab_size, embedding_dim, hidden_dim, output_dim).to(device)


In [ ]:
optimizer = optim.Adam(model.parameters())
criterion = nn.CrossEntropyLoss()

# Обучение модели
def train_model(model, iterator, optimizer, criterion):
    epoch_loss = 0
    model.train()
    for batch in iterator:
        optimizer.zero_grad()
        predictions = model(batch.text).squeeze(1)
        loss = criterion(predictions, batch.label)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
    return epoch_loss / len(iterator)

# Запуск обучения
num_epochs = 5
for epoch in range(num_epochs):
    train_loss = train_model(model, train_iterator, optimizer, criterion)
    print(f'Epoch {epoch+1}/{num_epochs} Loss: {train_loss:.3f}')


In [25]:
from catboost import CatBoostClassifier

def load_catboost_model(filename):
    model = CatBoostClassifier()
    model.load_model(filename, format='cbm')
    return model
# Загрузка модели из файла .cbm
model =load_catboost_model('catboost_model.cbm')

# Сохранение модели в формате JSON
model.save_model('model1.json', format='json')


CatBoostError: C:/Go_Agent/pipelines/BuildMaster/catboost.git/catboost/libs/model/model_export/model_exporter.cpp:152: Can save model with text features only in cbm format